In [23]:
import torch
import numpy as np
import pandas as pd
import random as rd
import losses as ls
import csv

In [24]:
# default stuff
dtype = torch.double
device = torch.device("cpu")

# architecture definition
N_in, N_hid, N_out = 2, 20, 4
batch_size = 200
learning_rate = 0.00001

#weight matrix
w1 = torch.randn(N_in, N_hid, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(N_hid, N_out, device=device, dtype=dtype, requires_grad=True)

In [25]:
# data manipulation
# ho cambiato il file così ora sono 0 e 1 i colori
data = pd.read_csv('/home/burrito/Documents/python/pytorch/Python-net/color_classifier/data_train_4.csv')
test = pd.read_csv('/home/burrito/Documents/python/pytorch/Python-net/color_classifier/data_test_4.csv')
data_np_x = data[['x', 'y']].to_numpy()
data_np_y = data[['color']].to_numpy()
data_test = test.to_numpy()

In [26]:
batchesX = np.asarray(np.array_split(data_np_x, 100))
batchesY = np.asarray(np.array_split(data_np_y, 100))

In [27]:
Xtest = torch.from_numpy(data_test)

In [28]:
for epoch in range(100000):
    b = rd.randint(0, 99)
    Xtr = torch.from_numpy(batchesX[b])
    Ytr = torch.from_numpy(batchesY[b])
    y = Xtr.mm(w1).clamp(min=0).mm(w2)
    # SVM loss
    # loss = ls.SVM_loss(y, Ytr)
    # cross entropy loss
    loss = ls.cross_entropy(y, Ytr)
    if epoch%10000 == 0:
        print(epoch, loss.item())
    loss.backward()
    with torch.no_grad():
        w1 -= learning_rate*w1.grad
        w2 -= learning_rate*w2.grad
        w1.grad.zero_()
        w2.grad.zero_()

0 183.28841878708985
10000 126.58338099525128
20000 125.56211121088333
30000 123.48003869856149
40000 121.84019439096868
50000 121.1234709665776
60000 120.72191773026857
70000 122.00388168165199
80000 120.7776748382796
90000 121.66749818499864


In [29]:
prediction = Xtest.mm(w1).clamp(min=0).mm(w2)

In [30]:
pred = open("/home/burrito/Documents/python/pytorch/Python-net/color_classifier/pred_4.csv", 'w+', newline='')
writer = csv.writer(pred)
writer.writerow(["color"])
for i in prediction:
    value, index = torch.max(i, 0)
    writer.writerow([index])